### DSWA + Visagio - Previsão de Demanda Diária

### Gabriel Varano, RA 822484
---

In [161]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

df_produto = pd.read_csv('produto.csv')
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [162]:
display(df_produto)

,SKU,NOME_PRODUTO,CATEGORIA,SUBCATEGORIA
0,3.0,COCO RALADO GROSSO KG ...,Doceria,Confeitaria
1,4.0,ICE TEA LEAO LATA 340ML ...,NaN,NaN
2,5.0,TAHINE ISTAMBUL 200G ...,Pelo Mundo,Pastas Árabes
3,6.0,AMENDOIM MOIDO KG ...,Castanhas & Oleaginosas,Oleaginosas moídas
4,7.0,HALAWI ISTAMBUL LATA 500G ...,Pelo Mundo,Pastas Árabes
...,...,...,...,...
6495,171874.0,BARRA BOLD BAR BANOFFEE 60G NV ...,Suplementos,Barrinhas
6496,171877.0,VINHO CASIL.D.CARNAVAL SPECTACULAR CARBENET 75...,Vinhos & Espumantes,Tintos
6497,171878.0,VINHO CASIL.D.CARNAVAL FANTASTIC SWEET 750ML C...,Vinhos & Espumantes,Tintos
6498,171887.0,BATATA PALHA EXTRA FINA VISCONTI 100G ...,Biscoitos,Batatas palhas


In [163]:
display(df_test)

,id,data,sku,cod_filial,filial,unidade
0,0,2024-12-02,9,101032,RUA,KG
1,1,2024-12-03,9,101032,RUA,KG
2,2,2024-12-04,9,101032,RUA,KG
3,3,2024-12-05,9,101032,RUA,KG
4,4,2024-12-06,9,101032,RUA,KG
...,...,...,...,...,...,...
2748,2748,2024-12-27,24706,101042,SHOPPING,KG
2749,2749,2024-12-28,24706,101042,SHOPPING,KG
2750,2750,2024-12-29,24706,101042,SHOPPING,KG
2751,2751,2024-12-30,24706,101042,SHOPPING,KG


In [164]:
display(df_train)

,data,sku,cod_filial,filial,unidade,demanda,faturamento,num_transacoes,clientes_unicos
0,2024-01-02,9,101032,RUA,KG,0.430,30.12,3,3
1,2024-01-03,9,101032,RUA,KG,0.238,16.65,2,2
2,2024-01-04,9,101032,RUA,KG,0.400,27.97,2,2
3,2024-01-05,9,101032,RUA,KG,0.378,26.45,2,2
4,2024-01-06,9,101032,RUA,KG,0.766,53.58,4,4
...,...,...,...,...,...,...,...,...,...
27574,2024-11-26,24706,101042,SHOPPING,KG,1.330,132.87,7,7
27575,2024-11-27,24706,101042,SHOPPING,KG,0.426,42.55,2,2
27576,2024-11-28,24706,101042,SHOPPING,KG,1.272,127.07,7,7
27577,2024-11-29,24706,101042,SHOPPING,KG,0.508,50.75,3,3


In [165]:
display(df_produto.isna().sum())

SKU                1
NOME_PRODUTO       1
CATEGORIA       2002
SUBCATEGORIA    2002
dtype: int64

In [166]:
df_produto = df_produto[:-1]
display(df_produto)

,SKU,NOME_PRODUTO,CATEGORIA,SUBCATEGORIA
0,3.0,COCO RALADO GROSSO KG ...,Doceria,Confeitaria
1,4.0,ICE TEA LEAO LATA 340ML ...,NaN,NaN
2,5.0,TAHINE ISTAMBUL 200G ...,Pelo Mundo,Pastas Árabes
3,6.0,AMENDOIM MOIDO KG ...,Castanhas & Oleaginosas,Oleaginosas moídas
4,7.0,HALAWI ISTAMBUL LATA 500G ...,Pelo Mundo,Pastas Árabes
...,...,...,...,...
6494,171803.0,CREME DE LEITE DOCE ECILA 35% GARRAFA 450G 302...,Refrigerados,Leites
6495,171874.0,BARRA BOLD BAR BANOFFEE 60G NV ...,Suplementos,Barrinhas
6496,171877.0,VINHO CASIL.D.CARNAVAL SPECTACULAR CARBENET 75...,Vinhos & Espumantes,Tintos
6497,171878.0,VINHO CASIL.D.CARNAVAL FANTASTIC SWEET 750ML C...,Vinhos & Espumantes,Tintos


In [167]:
display(df_produto.isna().sum())

SKU                0
NOME_PRODUTO       0
CATEGORIA       2001
SUBCATEGORIA    2001
dtype: int64

In [168]:
df_produto = df_produto.fillna("Sem Categoria")
df_produto["SKU"] = df_produto["SKU"].astype(int)
display(df_produto.isna().sum())
display(df_produto)

SKU             0
NOME_PRODUTO    0
CATEGORIA       0
SUBCATEGORIA    0
dtype: int64

,SKU,NOME_PRODUTO,CATEGORIA,SUBCATEGORIA
0,3,COCO RALADO GROSSO KG ...,Doceria,Confeitaria
1,4,ICE TEA LEAO LATA 340ML ...,Sem Categoria,Sem Categoria
2,5,TAHINE ISTAMBUL 200G ...,Pelo Mundo,Pastas Árabes
3,6,AMENDOIM MOIDO KG ...,Castanhas & Oleaginosas,Oleaginosas moídas
4,7,HALAWI ISTAMBUL LATA 500G ...,Pelo Mundo,Pastas Árabes
...,...,...,...,...
6494,171803,CREME DE LEITE DOCE ECILA 35% GARRAFA 450G 302...,Refrigerados,Leites
6495,171874,BARRA BOLD BAR BANOFFEE 60G NV ...,Suplementos,Barrinhas
6496,171877,VINHO CASIL.D.CARNAVAL SPECTACULAR CARBENET 75...,Vinhos & Espumantes,Tintos
6497,171878,VINHO CASIL.D.CARNAVAL FANTASTIC SWEET 750ML C...,Vinhos & Espumantes,Tintos


In [169]:
df_train['data'] = pd.to_datetime(df_train['data'])
df_test['data'] = pd.to_datetime(df_test['data'])

In [170]:
df_train = df_train.merge(df_produto, left_on='sku', right_on='SKU').drop(columns=['SKU', "NOME_PRODUTO", "faturamento", "num_transacoes", "clientes_unicos"])
df_test = df_test.merge(df_produto, left_on='sku', right_on='SKU').drop(columns=['SKU', 'NOME_PRODUTO'])

In [171]:
display(df_train)

,data,sku,cod_filial,filial,unidade,demanda,CATEGORIA,SUBCATEGORIA
0,2024-01-02,9,101032,RUA,KG,0.430,Castanhas & Oleaginosas,Castanhas de Caju
1,2024-01-03,9,101032,RUA,KG,0.238,Castanhas & Oleaginosas,Castanhas de Caju
2,2024-01-04,9,101032,RUA,KG,0.400,Castanhas & Oleaginosas,Castanhas de Caju
3,2024-01-05,9,101032,RUA,KG,0.378,Castanhas & Oleaginosas,Castanhas de Caju
4,2024-01-06,9,101032,RUA,KG,0.766,Castanhas & Oleaginosas,Castanhas de Caju
...,...,...,...,...,...,...,...,...
27574,2024-11-26,24706,101042,SHOPPING,KG,1.330,Frutas,Secas
27575,2024-11-27,24706,101042,SHOPPING,KG,0.426,Frutas,Secas
27576,2024-11-28,24706,101042,SHOPPING,KG,1.272,Frutas,Secas
27577,2024-11-29,24706,101042,SHOPPING,KG,0.508,Frutas,Secas


In [172]:
def caracteristicas_data(df):
    #df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.dayofweek
    #df['dia_mes'] = df['data'].dt.day
    #df['final_semana'] = df['dia_semana'].isin([5, 6]).astype(int)
    #df['inicio_mes'] = df['data'].dt.is_month_start.astype(int)
    return df

df_train = caracteristicas_data(df_train)
df_test = caracteristicas_data(df_test)

df_train = df_train.sort_values(["sku", "data"])
df_test = df_test.sort_values(["sku", "data"])

In [173]:
df_train["lag_1"] = df_train.groupby("sku")["demanda"].shift(1)
df_train["lag_7"] = df_train.groupby("sku")["demanda"].shift(7)
df_train["lag_14"] = df_train.groupby("sku")["demanda"].shift(14)
df_train["mm_7"] = df_train.groupby("sku")["demanda"].shift(1).rolling(7).mean()
df_train["mm_14"] = df_train.groupby("sku")["demanda"].shift(1).rolling(14).mean()
#df_train["std_7"] = df_train.groupby("sku")["demanda"].shift(1).rolling(7).std()

df_train = df_train.dropna().reset_index(drop=True)

stats_por_sku = df_train.groupby("sku").agg({
"demanda": ["last", "mean", "std"]
})
stats_por_sku.columns = ["last", "mean", "std"]
df_test["lag_1"] = df_test["sku"].map(stats_por_sku["last"])
df_test["lag_7"] = df_test["sku"].map(stats_por_sku["last"])
df_test["lag_14"] = df_test["sku"].map(stats_por_sku["last"])
df_test["mm_7"] = df_test["sku"].map(stats_por_sku["mean"])
df_test["mm_14"] = df_test["sku"].map(stats_por_sku["mean"])
#df_test["std_7"] = df_test["sku"].map(stats_por_sku["std"])

media = df_train["demanda"].mean()
preencher = ["lag_1", "lag_7", "lag_14", "mm_7", "mm_14"]
df_test[preencher] = df_test[preencher].fillna(media)

In [174]:
display(df_train)

,data,sku,cod_filial,filial,unidade,demanda,CATEGORIA,SUBCATEGORIA,dia_semana,lag_1,lag_7,lag_14,mm_7,mm_14
0,2024-01-11,9,101032,RUA,KG,0.580,Castanhas & Oleaginosas,Castanhas de Caju,3,0.766,1.592,0.430,1.040857,0.823000
1,2024-01-11,9,101042,SHOPPING,KG,1.323,Castanhas & Oleaginosas,Castanhas de Caju,3,0.580,0.766,0.258,0.896286,0.833714
2,2024-01-12,9,101032,RUA,KG,0.302,Castanhas & Oleaginosas,Castanhas de Caju,4,1.323,2.408,0.238,0.975857,0.909786
3,2024-01-12,9,101042,SHOPPING,KG,1.328,Castanhas & Oleaginosas,Castanhas de Caju,4,0.302,0.348,1.306,0.675000,0.914357
4,2024-01-13,9,101032,RUA,KG,0.420,Castanhas & Oleaginosas,Castanhas de Caju,5,1.328,1.284,0.400,0.815000,0.915929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26775,2024-11-28,24706,101042,SHOPPING,KG,1.272,Frutas,Secas,3,2.580,2.774,0.510,1.690286,1.591571
26776,2024-11-29,24706,101032,RUA,KG,3.694,Frutas,Secas,4,1.272,0.792,2.254,1.475714,1.646000
26777,2024-11-29,24706,101042,SHOPPING,KG,0.508,Frutas,Secas,4,3.694,2.072,1.290,1.890286,1.748857
26778,2024-11-30,24706,101032,RUA,KG,2.216,Frutas,Secas,5,0.508,1.330,3.474,1.666857,1.693000


In [175]:
display(df_test.sort_values(["data", "sku"]))

,id,data,sku,cod_filial,filial,unidade,CATEGORIA,SUBCATEGORIA,dia_semana,lag_1,lag_7,lag_14,mm_7,mm_14
28,28,2024-12-01,9,101042,SHOPPING,KG,Castanhas & Oleaginosas,Castanhas de Caju,6,4.244,4.244,4.244,2.069937,2.069937
85,85,2024-12-01,51,101042,SHOPPING,KG,Castanhas & Oleaginosas,Nozes,6,0.403,0.403,0.403,1.527893,1.527893
141,141,2024-12-01,65,101042,SHOPPING,KG,Castanhas & Oleaginosas,Castanhas do Pará,6,1.692,1.692,1.692,1.392724,1.392724
195,195,2024-12-01,66,101042,SHOPPING,KG,Castanhas & Oleaginosas,Castanhas do Pará,6,2.932,2.932,2.932,3.232769,3.232769
252,252,2024-12-01,86,101042,SHOPPING,KG,Matinais,Cereais,6,15.540,15.540,15.540,15.213369,15.213369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,2645,2024-12-31,23554,101042,SHOPPING,KG,Mixes da Casa,Mixes da Casa,1,4.726,4.726,4.726,2.879081,2.879081
2671,2671,2024-12-31,23980,101032,RUA,KG,Bacalhau & Pescados,Bacalhau,1,1.854,1.854,1.854,10.541425,10.541425
2697,2697,2024-12-31,23980,101042,SHOPPING,KG,Bacalhau & Pescados,Bacalhau,1,1.854,1.854,1.854,10.541425,10.541425
2723,2723,2024-12-31,24706,101032,RUA,KG,Frutas,Secas,1,2.156,2.156,2.156,1.294048,1.294048


In [176]:
X_train = df_train.drop(columns=["demanda"])
X_test = df_test.drop(columns=["id"])
y_train = df_train["demanda"]

In [177]:
display(X_train.dtypes)

data            datetime64[ns]
sku                      int64
cod_filial               int64
filial                  object
unidade                 object
CATEGORIA               object
SUBCATEGORIA            object
dia_semana               int32
lag_1                  float64
lag_7                  float64
lag_14                 float64
mm_7                   float64
mm_14                  float64
dtype: object

In [178]:
cat_features = ["sku", "cod_filial", "filial", "unidade", "CATEGORIA", "SUBCATEGORIA"]
for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

In [179]:
model = CatBoostRegressor(
    loss_function="RMSE",
    random_seed=111,
    verbose=False,
    cat_features=cat_features
)

param_grid = {
    'iterations': [1000, 2000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [4, 5, 6],
    'l2_leaf_reg': [5, 10, 20],
    'random_strength': [1, 2, 5],
    'bagging_temperature': [0, 0.5, 1]
}

result = model.randomized_search(
    param_grid,
    X=X_train,
    y=y_train,
    n_iter=15, 
    partition_random_seed=111,
    shuffle=True,
    stratified=False,
    verbose=False,
    plot=False
)

model.fit(
    X_train,
    y_train,
    early_stopping_rounds=200,
    verbose=200
)


bestTest = 3.669025606
bestIteration = 1169


bestTest = 3.673243058
bestIteration = 1942


bestTest = 3.705188013
bestIteration = 1998


bestTest = 3.660925892
bestIteration = 683


bestTest = 3.679794172
bestIteration = 821


bestTest = 3.640139058
bestIteration = 1180


bestTest = 3.655267681
bestIteration = 1604


bestTest = 3.668637179
bestIteration = 886


bestTest = 3.656950547
bestIteration = 1095


bestTest = 3.659214213
bestIteration = 1223


bestTest = 3.660925892
bestIteration = 683


bestTest = 3.685565424
bestIteration = 1999


bestTest = 3.647888058
bestIteration = 940


bestTest = 3.722970899
bestIteration = 252


bestTest = 3.656310595
bestIteration = 1340

Training on fold [0/3]

bestTest = 3.836115992
bestIteration = 1883

Training on fold [1/3]

bestTest = 3.245841757
bestIteration = 1843

Training on fold [2/3]

bestTest = 3.499167508
bestIteration = 551

0:	learn: 7.6891052	total: 57.9ms	remaining: 1m 55s
200:	learn: 3.3261244	total: 12.5s	remaining: 1m 51s
400:	

In [185]:
preds = model.predict(X_test)
y_final = preds

submission = pd.DataFrame({
    "id": df_test["id"],
    "demanda": y_final
})
submission["demanda"] = submission["demanda"].clip(lower=0)

submission.to_csv("submission.csv", index=False)

In [181]:
display(submission.iloc[1106])

id         1105.000000
demanda       3.554276
Name: 1105, dtype: float64

In [182]:
importances = model.get_feature_importance()
feature_names = X_train.columns

fi = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

display(fi)

,feature,importance
11,mm_7,23.860062
12,mm_14,20.006152
4,unidade,10.539621
3,filial,6.639138
0,data,6.451047
2,cod_filial,6.076245
5,CATEGORIA,5.625085
7,dia_semana,4.402810
10,lag_14,4.285121
8,lag_1,4.074309
